### THIS IS A SIMPLE COUPLING FRAMEWORK OF KU and ECSimpleSnow MODELS

### - Ku model is originally developed in Python, ANNUAL TIME STEP
### - ECSimpleSnow is developed in Fortran, DAILY TIME STEP

### This notebook will be aimed at setting up a scenario model for a river floodplain
### The river floodplain has a main channel 

#### *** ECSimpleSnow model 
#### - uses daily air temperature and precipitation as main inputs; 
#### - provides mean annual air temperature, annual amplitude of air temperature, winter-averged snow depth and density for ku model 

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

# Load PyMT model(s)
import pymt.models
# ku = models.
ku = pymt.models.Ku()
ec = pymt.models.ECSimpleSnow()

➡ models: FrostNumber, Ku, Hydrotrend, ECSimpleSnow, Cem, Waves


### This function is used to estimate the annual temperature curve:

In [2]:
def func(x, a, b, c):
    
    # a: annual amplitude
    # c: mean annual temperature
    
    return a * np.sin(x/365*2*np.pi+b) + c

### Initialize ECSimpleSnow component

In [3]:
# Initialize the model with the defaults.
ec.initialize('snow_model.cfg')

# List input and output variable names.
print(ec.get_output_var_names())
print(ec.get_input_var_names())
ec.finalize()

('snowpack__depth', 'snowpack__mass-per-volume_density')
('precipitation_mass_flux', 'land_surface_air__temperature', 'precipitation_mass_flux_adjust_factor', 'snow_class', 'open_area_or_not', 'snowpack__initial_depth', 'snowpack__initial_mass-per-volume_density')


### initialize KU component

In [ ]:
# We create a large time step here, it could be able to update ku model for many times with once initialization.
# It will use the same soil properties.

config_file, run_folder = ku.setup(lat  = 71.31,
                                   lon  = -156.66,
                                   T_air = -10.0,
                                   A_air = 16,
                                   end_year = 3000)

ku.initialize(config_file, run_folder)
print(ku.get_component_name())
print(ku.get_input_var_names())

print(ku.get_value('vegetation__Hvgf'))
ku.finalize()

 
Ku model component: Initializing...
Permamodel Ku Component
('latitude', 'longitude', 'datetime__start', 'datetime__end', 'atmosphere_bottom_air__temperature', 'atmosphere_bottom_air__temperature_amplitude', 'snowpack__depth', 'snowpack__density', 'water-liquid__volumetric-water-content-soil', 'vegetation__Hvgf', 'vegetation__Hvgt', 'vegetation__Dvf', 'vegetation__Dvt')
[ 0.]
NO OUTPUT of ALT
NO OUTPUT of TPS
***
Writing output finished!
Please look at./off.nc and ./off.nc


## Define variables to save results for plotting ...

In [ ]:
site_id    = np.arange(12)
site_snod  = np.zeros(12) # winter-averaged snow depth
site_vht   = np.zeros(12) # thaw season veg height
site_vhf   = np.zeros(12) # freeze season veg height
site_ALT   = np.zeros(12) # active layer thickness
site_TPS   = np.zeros(12) # mean annual temperature at permafrost surface
site_fst   = np.zeros(12) # forested=1

print(site_snod)

[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]


### Run

In [ ]:
ku.initialize(config_file, run_folder)
    
for i in np.arange(12):
    
    # This is an unknow issue: we have to reset the module objective here. 
    # It will be double checked.
    
    # ec = pymt.models.ECSimpleSnow()   
    ec.initialize('snow_model.cfg')
            
    # DEFINE some variables for annual summary

    snod     = 0
    sden     = 0
    snow_day = 0
    
    t365 = np.arange(365)
    air_temperature = np.zeros(365) # this is used for estimating annual cycle in following
    
    print('setting column - ',np.str(i))
    
#     # set different vegetation height, snow class, forested area for each site:

    if i == 1 or i == 0:      
        
        ku.set_value('vegetation__Hvgt', 2.0)
        ku.set_value('vegetation__Hvgf', 2.0)
        ku.set_value('vegetation__Dvt' , 1.0E-8)
        
        site_vht[i] = 2.0
        site_vhf[i] = 2.0
                
        #ec.set_value('snow_class', 2) # snow class 2 is typical for taiga areas 
        ec.set_value('open_area_or_not', 0) # 0 means we have a taiga forest
        
        site_fst[i] = 1 - 0        
        
#     if i == 2 or i == 3 or i ==7:
    
#         ku.set_value('vegetation__Hvgt', 1.0)
#         ku.set_value('vegetation__Dvt' , 1.0E-8)
        
#         site_vht[i] = 1.0
        
#         ec.set_value('snow_class'      , 1) # snow class 2 is typical for taiga areas 
#         ec.set_value('open_area_or_not', 1) # 0 means we have a taiga forest
        
#         site_fst[i] = 1 - 1
        
#     if i == 4 or i == 5 or i==6:
    
#         ku.set_value('vegetation__Hvgt', 0.0)
#         ku.set_value('vegetation__Dvt' , 1.0E-8)
        
#         site_vht[i] = 0.0
        
#         ec.set_value('snow_class'      , 1) # snow class 2 is typical for taiga areas 
#         ec.set_value('open_area_or_not', 1) # 0 means we have a taiga forest
        
#         site_fst[i] = 1 - 1
            
#     if i == 8 or i==9:
#         ku.set_value('vegetation__Hvgt', 0.1)
#         ku.set_value('vegetation__Dvt' , 1.0E-8)
        
#         site_vht[i] = 0.1
        
#         ec.set_value('snow_class', 1) # snow class 2 is typical for taiga areas 
#         ec.set_value('open_area_or_not', 1) # 0 means we have a taiga forest
        
#         site_fst[i] = 1 - 1
                
    # loop for 365 days to drive snow simulation:
            
    for iii in np.arange(365):
        
        # please note that: set_value for BMI-Fortran is sensitive to variable type.
        # It's easy to get error.
                
        #  ec.set_value('land_surface_air__temperature', np.float32(air_temperature[iii]))
        #  ec.set_value('precipitation_mass_flux'      , np.float32(precipitation[iii]))
        
        ec.update()
        
        tair  = ec.get_value('land_surface_air__temperature')
        snd   = ec.get_value('snowpack__depth', units='m')
        rsn   = ec.get_value('snowpack__mass-per-volume_density')

        units = ec.get_var_units('snowpack__depth')

        snod = snod + snd
        sden = sden + rsn
        
        air_temperature[iii] = tair

        if tair <= 0:
            snow_day = snow_day + 1

    # Calculate winter-averaged snow depth and density:

    SDEN = sden / snow_day 
    SNOD = snod / snow_day

    # Finalize the snow process:
    ec.finalize()    
        
    # Estimate mean and amplitude from daily air temperature:
    
    popt, pcov = curve_fit(func, 
                           t365, 
                           air_temperature, 
                           bounds=([0.0,-99.0,-99.0], [50., 99.0, 99.0]))

    MAAT = popt[2]
    MAAA = popt[0]
    
    # Set values for Ku component:
    
    ku.set_value('atmosphere_bottom_air__temperature'          , MAAT)
    ku.set_value('atmosphere_bottom_air__temperature_amplitude', MAAA)
    ku.set_value('snowpack__depth'                             , SNOD)
    ku.set_value('snowpack__density'                           , SDEN)
    ku.set_value('water-liquid__volumetric-water-content-soil' , 0.45)
    
    # Force Ku model for once: [i.e., by 1 column]
           
    ku.update()
    
    alt = ku.get_value('soil__active_layer_thickness')
    tps = ku.get_value('soil__temperature')    
    
    site_snod[i] = SNOD
    site_ALT[i]  = alt
    site_TPS[i]  = tps
    
ku.finalize()

 
Ku model component: Initializing...


### Quick Show Results:

In [ ]:
plt.plot(site_snod, 'o-')
plt.ylabel('Snow Depth (m)')

In [ ]:
plt.plot(site_fst, 'o-')
plt.ylabel('Forested=1')

In [ ]:
plt.plot(site_vht, 'o-')
plt.ylabel('Veg. H. Thaw (m)')

In [ ]:
plt.plot(site_ALT, 'o-')
plt.ylim([0.4, 0])
plt.ylabel('ALT (m)')

In [ ]:
plt.plot(site_TPS, 'o-')
plt.ylabel('Mean Annual Temperature at PF Surface')

In [ ]:
a = [0,]
type(a)